In [2]:
import re

import tensorflow as tf

# optimizer : 학습속도를 빠르고 안정적이게 하는 방법, 오차의 최저점을 찾아가는것
#Adam Optimizer에서는 두 개의 momentum을 활용한다. m, v
#m : 기존 momentum 방식에서 활용하던대로 Gradient 값을 좀 더 빠르게 계산할 수 있도록 돕는 역활
# v : 데이터의 분포가 Sparse한 곳에서 그 영향력을 극대화 시킴으로써 빠르게 sparse한 영역을 벗어날 수 있도록 돕는 역활
# m, v를 사용하여 모멘텀 효과와 weight마다 다른 learning rate를 적용하는 adaptive rate효과를 동시에 보는 알고르즘


In [3]:
class WarmUp(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_schedule_fn, warmup_steps, power=1.0, name=None):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_steps = warmup_steps
        self.power = power
        self.decay_schedule_fn = decay_schedule_fn
        self.name = name
        
    def __call__(self, step):
        with tf.name_scope(self.name or "WarmUp") as name:
            #neural network의 노드가 많을경우 쉽게 볼수 없기 때문에 단순화 작업이 필요
            #구문을 사용해서 레이어 이름별로 범위를 나눠주는 과정
            #name scope를 지정하면 계층 구조의 맨 위만 표시
            # warm up : 처음에는 reconstruction error만을 이용해 학습하고, 학습이 진행될수록 loss함수에서 KL
            #KL-divergence 의 비중을 늘려주는 것
            global_step_float = tf.cast(step, tf.float32) #그래프의 훈련횟수(the number of batches)
            warmup_step_float = tf.cast(self.warmup_steps, tf.float32) # a few updates with low learning rate before/ at the beginning of training
            warmup_percent_done = global_step_float / warmup_steps_float
            warmup_learning_rate = self.initial_learning_rate * tf.math.pow(warmup_percent_done, self.power)
            return tf.cond(
                global_step_float < warmup_steps_float,  
                lambda: warmup_learning_rate,
                lambda: self.decay_schedule_fn(step),
                name=name,
            )# (추측)warmup step의 개수는 작은 learning rate를 가질때 발생하는 횟수 이므로 global step보다 클수밖에 없다라고 생각!
    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "decay_schedule_fn": self.decay_schedule_fn,
            "warmup_steps": self.warmup_steps,
            "power": self.power,
            "name": self.name,
        }
    
def create_optimizer(init_lr, num_train_steps, num_warmup_steps):
    learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=init_lr, decay_steps=num_train_steps, end_learning_rate=0.0
    )
    if num_warmup_steps:
        learning_rate_fn = WamUp(
        learning_rate=learning_rate_fn,
        weight_decay_rate=0.01,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-6,
        exclude_from_weight_decay=["layer_norm", "bias"],
        )
        return optimizer

In [8]:
class AdamWeightDecay(tf.keras.optimizers.Adam):
    def __init__(
        self,
        learning_rate = 0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7,
        amsgrad=False,
        weight_decay_rate=0.0, #0보다 크거나 같은 float 값. 업데이트마다 적용되는 학습률의 감소율
        include_in_weight_decay=None,
        exclude_from_weight_decay=None,
        name="AdamWeightDecay",
        **kwargs
    ):
        super().__init__(learning_rate, bata_1, bata_2, epsilon, amsgrad, name, **kwargs)
        self.weight_decay_rate = weight_decay_rate
        self._include_in_weight_decay = include_in_weight_decay
        self._exclude_from_weight_decay = exclude_from_weight_decay

        def from_config(cls, config): #config with WarmUp custom object
            custom_objects = {"WarmUp": WarmUp}
            return super().from_config(config, custom_objects=custom_objects)
        
        def _prepare_local(self, var_device, var_dtype, apply_state): #var_device, var_dtype : 정확하게 어떻게사용하는지 모르겠음!
            super()._prepare_local(var_device, var_dtype, apply_state)
            apply_state["weight_decay_rate"] = tf.constant(self.weight_decay_rate, name="adam_weight_decay_rate")
            
        def _decay_weight_op(self, var, learning_rate, apply_state):
            do_decay = self._do_use_weight_decay(var.name)
            if do_decay:
                return var.assign_sub(
                    learning_rate * var * apply_state["weight_decay_rate"], use_locking=self._use_locking
                )#사진 참고한거 보면 : gradient descent 구하는 방식으로 추측됨. 원문 코드 보면 next parameter = parameter - learning rate * parameter * weight decay(기울기) 라고 표현
            return tf.no_op() #does nothing
        

        def apply_gradients(self, grads_and_vars, clip_norm, name=None):
            grads, tvars = list(zip(*grads_and_vars))
            #grads : tf.gradients(loss, tvars) 이므로 loss를 tvars로 미분하는 과정(loss = 기울기 * tvars + 상수항) 그러니까 기울기를 뜻함
            #tvars : tf.trainable_variable() 이므로 trainable(bool 형식)이 True일 때 , variable을 학습을 통해 값을 변화시킬것인지
            (grads, _) = tf.clip_by_global_norm(grads, clip_norm=clip_norm) # how the model was pre-trained
            return super().apply_gradients(zip(grads, tvars))
        
        def _get_lr(self, var_device, var_dtype, apply_state):
            #주어진 state를 가지고 learning rate 검색
            if apply_state is None:
                return self._decayed_lr_t[var_dtype], {}
            
            apply_state = apply_state or {}
            coefficients = apply_state.get((var_device, var_dtype)) #get(key, value)로 구성되어있으면 key를 통해 value값을 뽑아낸다
            if coefficients is None:
                coefficients = self._fallback_apply_state(var_device, var_dtype)
                apply_state[(var_device, var_dtype)] = coefficients
                
            return coefficients["lr_t"], dict(apply_state = apply_state)
        
        def _resource_apply_dense(self, grad, var, apply_state=None):
            lr_t, kwargs = self._get_lr(var.device, var.dtype.base_dtype, apply_state)
            decay = self._decay_weights_op(var, lr_t, apply_state)
            with tf.control_dependencies([decay]): #연산간의 실행 순서를 정해주는 역활
                return super()._resource_apply_dense(grad, var, **kwargs)
            
        def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
            lr_t, kwargs = self._get_lr(var.device, var.dtype.base_dtype, apply_state)
            decay = self._decay_weights_op(var, lr_t, apply_state)
            with tf.control_dependencies([decay]):
                return super()._resource_apply_sparse(grad, var, indices, **kwargs)
            
        def get_config(self):
            config = super().get_config()
            config.update({"weight_decay_rate: self.weight_decay_rate"})
            return config
        
        def _do_use_weight_decay(self, param_name):
            if self.weight_decay_rate ==0:
                return False
            
        if self._include_in_weight_decay:
            for r in self._include_in_weight_decay:
                if re.search(r, param_name) is not None:
                    return True

        if self._exclude_from_weight_decay:
            for r in self._exclude_from_weight_decay:
                if re.search(r, param_name) is not None:
                    return False
        return True

In [9]:
class GradientAccumulator(object):
    #SGD를 활용할 경우, gradient에 비해서 오차(Variance)가 존재할 수 있다. 이 때 최적화를 수행할때 발생하는 문제가
    # Noisy Gradient Problem이라고 부른다. 이런 문제를 해결하기 위해 두가지 방법을 사용
    #1) 모멘텀을 사용하지 않는 옵티마이저를 사용하여 안정적인 학습을 진행했으나, 수렴도가 만족스럽지 않는것
    #2) WarmUp을 사용하여 안정적인 학습과 수렴정도도 만족스러웠으나, 하이퍼파라미터에 민감하다는 단점
    #그래서 모멘텀을 사용하는 옵티마이저를 활용하면서 하이퍼파라미터에 상대적으로 덜 민감한 방법에 대해 고민한 끝에 Large Batch Size를
    # 사용하여 학습이 진행되는 중에 발생하는 Noisy Gradient가 경감되는 것을 확인. 
    # 단순히 Batch Size를 키우는것도 좋지만, Gpu 의 Memory가 한정적이기때문에 한정된 Gpu Memory내에서 Batch Size를 키우는 효과를 내기위해
    # Gradient Accumulation 방법 사용. 위 방법은 Step마다 파라미터를 업데이트 하지 않고, Gradient를 모으다가 일정한 수의 Gradient Vector들이 모이면 파라미터를 업데이트 하는 형식
    """Distribution strategies-aware gradient accumulation utility."""

    def __init__(self):
        """Initializes the accumulator."""
        self._gradients = []
        self._accum_steps = tf.Variable(
            initial_value=0, dtype=tf.int64, trainable=False, aggregation=tf.VariableAggregation.ONLY_FIRST_REPLICA
        )

    @property
    def step(self):
        """Number of accumulated steps."""
        return self._accum_steps.value()

    @property
    def gradients(self):
        """The accumulated gradients."""
        return list(
            gradient.value() if gradient is not None else gradient for gradient in self._get_replica_gradients()
        )

    def __call__(self, gradients):
        """Accumulates :obj:`gradients`."""
        if not self._gradients:
            self._gradients.extend(
                [
                    tf.Variable(tf.zeros_like(gradient), trainable=False) if gradient is not None else gradient
                    for gradient in gradients
                ]
            )

        if len(gradients) != len(self._gradients):
            raise ValueError("Expected %s gradients, but got %d" % (len(self._gradients), len(gradients)))

        for accum_gradient, gradient in zip(self._get_replica_gradients(), gradients):
            if accum_gradient is not None:
                accum_gradient.assign_add(gradient)

        self._accum_steps.assign_add(1)

    def reset(self):
        """Resets the accumulated gradients."""
        if self._gradients:
            self._accum_steps.assign(0)

        for gradient in self._get_replica_gradients():
            if gradient is not None:
                gradient.assign(tf.zeros_like(gradient))

    def _get_replica_gradients(self):
        if tf.distribute.has_strategy():
            # In a replica context, we want to accumulate gradients on each replica
            # without synchronization, so we directly assign the value of the
            # current replica.
            replica_context = tf.distribute.get_replica_context()

            if replica_context is None or tf.distribute.get_strategy().num_replicas_in_sync == 1: #replica_context 값이 없거나, gpu 장치 개수가 1개인경우
                return self._gradients 

            return (
                gradient.device_map.select_for_current_replica(gradient.values, replica_context)
                for gradient in self._gradients
            )
        else:
            return self._gradients